In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def modulator(bitseq, b):
    N_bits = len(bitseq)
    if b == 1:
        sym_table = np.exp(1j * [0, -np.pi])
        sym_table = sym_table[[1, 0]]
        inp = bitseq
        mod_symbols = sym_table[inp]
        M = 2
    elif b == 2:
        sym_table = np.exp(1j * np.pi / 4 * [-3, 3, 1, -1])
        sym_table = sym_table[[0, 1, 3, 2]]
        inp = np.reshape(bitseq, (b, N_bits // b))
        mod_symbols = sym_table[[2, 1] @ inp]
        M = 4
    elif b == 3:
        sym_table = np.exp(1j * np.pi / 4 * np.arange(8))
        sym_table = sym_table[[0, 1, 3, 2, 6, 7, 5, 4]]
        inp = np.reshape(bitseq, (b, N_bits // b))
        mod_symbols = sym_table[[4, 2, 1] @ inp]
        M = 8
    elif b == 4:
        m = 0
        sq10 = np.sqrt(10)
        sym_table = []
        for k in range(-3, 4, 2):
            for l in range(-3, 4, 2):
                m += 1
                sym_table.append((k + 1j * l) / sq10)
        sym_table = np.array(sym_table)[[0, 1, 3, 2, 4, 5, 7, 6, 12, 13, 15, 14, 8, 9, 11, 10]]
        inp = np.reshape(bitseq, (b, N_bits // b))
        mod_symbols = sym_table[[8, 4, 2, 1] @ inp]
        M = 16
    else:
        raise ValueError('Unimplemented modulation')

    return mod_symbols, sym_table, M

# Simulation Parameters
L_frame = 130
N_packet = 4000
b = 3
SNRdBs = np.arange(0, 21, 2)
sq2 = np.sqrt(2)

# MIMO Parameters
for iter in range(3):
    if iter == 0:
        NT, NR, gs = 1, 1, '-kx'  # SISO
    elif iter == 1:
        NT, NR, gs = 1, 2, '-^'  # Numbers of Tx/Rx antennas
    else:
        NT, NR, gs = 1, 4, '-ro'

    sq_NT = np.sqrt(NT)
    BER = np.zeros(len(SNRdBs))

    for i_SNR in range(len(SNRdBs)):
        SNRdB = SNRdBs[i_SNR]
        sigma = np.sqrt(0.5 / 10**(SNRdB / 10))
        noeb_p = np.zeros(N_packet)

        for i_packet in range(N_packet):
            symbol_data = np.random.randint(0, L_frame * b, NT)
            bit_data = np.reshape(np.transpose(np.unpackbits(np.array(symbol_data, dtype=np.uint8))), (NT * b, 1))
            temp, sym_tab, P = modulator(bit_data, b)
            X = temp.T
            Hr = (np.random.randn(L_frame, NR) + 1j * np.random.randn(L_frame, NR)) / sq2
            H = np.reshape(Hr, (L_frame, NR))
            Habs = np.sum(np.abs(H)**2, axis=1)
            Z = 0

            R = np.zeros((L_frame, NR), dtype=np.complex128)
            for i in range(NR):
                R[:, i] = np.sum(H[:, i] * X, axis=1) / sq_NT + sigma * (np.random.randn(L_frame) + 1j * np.random.randn(L_frame))
                Z = Z + R[:, i] * np.conj(H[:, i])

            d1 = np.abs(np.sum(Z, axis=1) - sym_tab)**2 + (-1 + np.sum(Habs)) * np.abs(sym_tab)**2
            i1 = np.argmin(d1, axis=1)
            Xd = sym_tab[i1].conj().T
            temp1 = X > 0
            temp2 = Xd > 0
            noeb_p[i_packet] = np.sum(temp1 != temp2)

        BER[i_SNR] = np.sum(noeb_p) / (N_packet * L_frame * b)

    plt.semilogy(SNRdBs, BER, gs)

plt.title('BER performance of MRC Scheme')
plt.xlabel('SNR [dB]')
plt.ylabel('BER')
plt.grid(True)
plt.legend(['SISO', 'MRC (Tx:1, Rx:2)', 'MRC (Tx:1, Rx:4)'])
plt.show()


In [2]:
import numpy as np

def bpsk_modulator(bitseq):
    sym_table = np.exp(1j * [0, -np.pi])
    sym_table = sym_table[[1, 0]]
    mod_symbols = sym_table[bitseq]
    M = 2
    return mod_symbols, sym_table, M

def qpsk_modulator(bitseq):
    sym_table = np.exp(1j * np.pi / 4 * [-3, 3, 1, -1])
    sym_table = sym_table[[0, 1, 3, 2]]
    inp = np.reshape(bitseq, (2, len(bitseq) // 2))
    mod_symbols = sym_table[[2, 1] @ inp]
    M = 4
    return mod_symbols, sym_table, M

def eight_psk_modulator(bitseq):
    sym_table = np.exp(1j * np.pi / 4 * np.arange(8))
    sym_table = sym_table[[0, 1, 3, 2, 6, 7, 5, 4]]
    inp = np.reshape(bitseq, (3, len(bitseq) // 3))
    mod_symbols = sym_table[[4, 2, 1] @ inp]
    M = 8
    return mod_symbols, sym_table, M

def sixteen_qam_modulator(bitseq):
    m = 0
    sq10 = np.sqrt(10)
    sym_table = []
    for k in range(-3, 4, 2):
        for l in range(-3, 4, 2):
            m += 1
            sym_table.append((k + 1j * l) / sq10)
    sym_table = np.array(sym_table)[[0, 1, 3, 2, 4, 5, 7, 6, 12, 13, 15, 14, 8, 9, 11, 10]]
    inp = np.reshape(bitseq, (4, len(bitseq) // 4))
    mod_symbols = sym_table[[8, 4, 2, 1] @ inp]
    M = 16
    return mod_symbols, sym_table, M


In [3]:
bitseq = np.array([0, 1, 0, 1, 1, 0, 1, 0])  # Replace with your bit sequence

mod_symbols, sym_table, M = qpsk_modulator(bitseq)
print("Modulated Symbols:", mod_symbols)
print("Symbol Table:", sym_table)
print("Constellation Size (M):", M)


TypeError: can't multiply sequence by non-int of type 'complex'

In [4]:
import numpy as np

def qpsk_modulator(bitseq):
    sym_table = np.exp(1j * np.pi / 4 * [-3, 3, 1, -1])
    sym_table = sym_table[[0, 1, 3, 2]]
    inp = np.reshape(bitseq, (2, len(bitseq) // 2))
    mod_symbols = sym_table[[2, 1] @ inp]
    M = 4
    return mod_symbols, sym_table, M

# Given bit sequence
bitseq = np.array([0, 1, 0, 1, 1, 0, 1, 0])

# Use the QPSK modulator function
mod_symbols, sym_table, M = qpsk_modulator(bitseq)

# Display the results
print("Given Bit Sequence:", bitseq)
print("Modulated Symbols:", mod_symbols)
print("Symbol Table:", sym_table)
print("Constellation Size (M):", M)


TypeError: can't multiply sequence by non-int of type 'complex'